# Lesson 1 - New tools for Data Scientists

In this first lesson we will see the main tools of the Anaconda environment for data analysis and code development; then we will learn how to read from and write to different kind of files.

## Anaconda

Anaconda (https://www.anaconda.com/) is a free platform for Python usage, inclusive of:
- a package-management system, called _conda_
- a series of tools for Python programming, all accessible from _Anaconda Navigator_ interface
- a development environment with all the most important packages for scientific programming.

#### Conda
Conda (https://docs.conda.io/projects/conda/en/latest/#) is an open source system for package and environment mangement. We can use it for:
- create a new environment, defined by a Python version and a series of installed packages (each with a specific version)<br>
    *conda create --name env_name python=version_num package_1 package_2*
- clone an existing environment, to replicate the results of a project<br>
    *conda create --name env_name -f=packages_file.yaml*
- install, uninstall, update all the Python packages present on all the distribution channels<br>
    *conda install package*<br>
    *pip install package*

The default active environment is _base_. If we want to activate (use) another one:<br>
    *conda activate env_name*

#### Anaconda Navigator
With this interface we can:
- open tools like Jupyter Notebook, Spyder and QTConsole
- manage packages and environments in a more "visual" way
- find links to documentation and community

## Import dati

To read, write, explore and process data we will use _pandas_ libray (https://pandas.pydata.org/): it is the _de facto_ framework to use data in Python.<br>

![title](./img/pandas_logo.png)

In [ ]:
import pandas as pd

The base structure we will use is the _DataFrame_, a 2-dimension table (rows x columns) to represent "structured" data.

In [ ]:
pd.DataFrame({'col_1': ['A', 'B', 'C', 'B'], 'col_2': [1, 5, 7, 3]})

The leftmost number is the __index__: it is an univocal number assigned to each record. It's the column name equivalent for rows.<br>
We can use an existing column as index, using the parameter *index_col* while reading data or the function *set_index* in any moment; the default index is a progressive integer starting from 0.

### CSV

**Input --- read_csv:** https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_csv.html#pandas.read_csv

Main parameters:
- _sep_: symbol to separate data, default ','
- *use_cols*: list of columns to import; if not specified, it imports all the columns
- _skiprows_ & *n_rows*: list of rows (or number of rows) to skip / number of rows to read; if not specified, it imports all the data
- *na_values*: list to values equivalent to **missing**
- *parse_dates*: list of columns to treat as dates
- _dtype_: dictionary of columns types 

In [ ]:
df_base = pd.read_csv('./Data/input/base_table.csv')

Fundamental functions/attributes to analyse the dataset:
- _head(n)_: first _n_ rows
- _tail(n)_: last _n_ rows
- _dtypes_: column types
- _shape_: dataset dimension (rows, columns)

In [ ]:
df_base.head()

In [ ]:
df_base.shape

In [ ]:
df_base.dtypes

The _object_ columns contain categorical / mixed data.
We can notice that *loan_dt* and *birth_dt* are NOT automatically recognised as dates. Moreover, customer NDG is seen as integer and not code (_str_).

#### _Exercise_
re-read the file using *parse_dates* e _dtype_ parameters to correct the issues.

#### _Exercise_
Import the file of the bank account balances, *account_bal.csv*, into the dataframe *df_balance*. The output dataframe must have the correct data types.<br>_Tip: pay attention to sep parameter_

**Output --- to_csv:** https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.to_csv.html

Main parameters:
- *columns*: list of columns to export; if not specified, it exports all the columns
- *index*: if **True** (as default), the index is written in output file; if **False** the index is omitted

In [ ]:
df_base.to_csv('./Data/file_prova.csv', index=False, sep=';')

### Excel

**Input --- read_excel:** https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_excel.html

Main parameters:
- all from *read_csv*
- *sheet_name*: name of the sheet from which read the data; we can also set a number (starting from 0). If not specified, it reads the first sheet.

To analyse the Excel file without opening it, we can use the _ExcelFile_ class.

In [ ]:
excelfile = pd.ExcelFile('./Data/input/input_data.xlsx')

print(excelfile.sheet_names)

In [ ]:
df_services = pd.read_excel('./Data/input/input_data.xlsx',
                            sheet_name='service_registry',
                            parse_dates=['serv_start_dt', 'serv_end_dt'],
                            dtype={'NDG': str})

df_services.head()

**Output --- to_excel:** https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.to_excel.html

Main parameters:
- all from *to_csv*
- *sheet_name*: name of output sheet.

Combining *to_excel* with *ExcelWriter* class, we can write several sheets without overwriting.

In [ ]:
writer = pd.ExcelWriter('./excel_foo.xlsx')

In [ ]:
df_services.to_excel(writer, sheet_name='registry', index=False)

df_foo = pd.DataFrame({'col_1': ['A', 'B', 'C', 'B'], 'col_2': [1, 5, 7, 3]})
df_foo.to_excel(writer, sheet_name='foo', index=False)

In [ ]:
writer.close()

We have to close the file object to use it.

### SQL

We will read data from a _sqlite_ database, so we will use the _sqlite3_ library to create a connection.

In [2]:
import sqlite3

conn = sqlite3.connect('./Data/input/input_data.db')

**Input --- read_sql:** https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_sql.html

Main parameters:
- the first parameter (_sql_) can be the input table name __or__ an SQL query
- the second parameter (_con_) is a connection to database.

reading the *sqlite_master* table we can extrapolate all the info of the database and its tables.

In [ ]:
pd.read_sql('select * from sqlite_master', engine)

In [ ]:
df_mov = pd.read_sql('account_mov', engine)

#### _Exercise_
Import *loans_data* table into dataframe *df_loans*, excluding the rows where *loan_type* is equal to UNSECURED value.

To write to database, we can use the same methodologies.

**Output --- to_sql:** https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.to_sql.html

In [ ]:
df_loans.to_sql('loans_secured', engine, index=False)